In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pytorch_lightning as pl
from typing import Sequence
import squidpy as sq
import torch
import pandas as pd
from torch_geometric.data import Data
from anndata import AnnData
from gpu_spatial_graph_pipeline.utils import adata2data
from gpu_spatial_graph_pipeline.data.datamodule import GraphAnnDataModule
from gpu_spatial_graph_pipeline.models.linear_ncem import LinearNCEM

In [3]:
#Mibitof
# adata = sq.datasets.mibitof()
# #feature_name=adata.obs.keys()[0] #Use for IMC dataset
# feature_name='Cluster'
# num_features=len(set(adata.obs[feature_name]))
# encoder_hidden_dims=30
# decoder_hidden_dims=30
# num_vars=adata.X.shape[1]

#IMC


adata = sq.datasets.imc()
feature_name=adata.obs.keys()[0] #Use for IMC dataset

def adata2data_imc(adata: AnnData):
    return adata2data(adata,feature_name)

num_features=len(set(adata.obs[feature_name]))
num_vars=adata.X.shape[1]


In [4]:
dm = GraphAnnDataModule(adata=adata,adata2data_fn=adata2data_imc, num_workers = 8, batch_size=100)

In [5]:
dm.setup()

In [6]:
for batch in dm.train_dataloader():
    print(batch)

Data(num_nodes=52, edge_index=[2, 2], x=[52, 11], y=[52, 34])
Data(num_nodes=37, edge_index=[2, 1], x=[37, 11], y=[37, 34])
Data(num_nodes=46, edge_index=[2, 4], x=[46, 11], y=[46, 34])
Data(num_nodes=52, edge_index=[2, 2], x=[52, 11], y=[52, 34])
Data(num_nodes=47, edge_index=[2, 5], x=[47, 11], y=[47, 34])
Data(num_nodes=41, edge_index=[2, 3], x=[41, 11], y=[41, 34])
Data(num_nodes=55, edge_index=[2, 6], x=[55, 11], y=[55, 34])
Data(num_nodes=43, edge_index=[2, 4], x=[43, 11], y=[43, 34])
Data(num_nodes=51, edge_index=[2, 2], x=[51, 11], y=[51, 34])
Data(num_nodes=53, edge_index=[2, 3], x=[53, 11], y=[53, 34])
Data(num_nodes=30, edge_index=[2, 0], x=[30, 11], y=[30, 34])
Data(num_nodes=50, edge_index=[2, 5], x=[50, 11], y=[50, 34])
Data(num_nodes=54, edge_index=[2, 3], x=[54, 11], y=[54, 34])
Data(num_nodes=38, edge_index=[2, 1], x=[38, 11], y=[38, 34])
Data(num_nodes=50, edge_index=[2, 4], x=[50, 11], y=[50, 34])
Data(num_nodes=50, edge_index=[2, 0], x=[50, 11], y=[50, 34])
Data(num

In [7]:
model = LinearNCEM(in_channels=num_features,out_channels=num_vars, lr=0.0001,weight_decay=0.000001)

In [8]:
gpu=False
if gpu:
    trainer:pl.Trainer = pl.Trainer(accelerator='gpu',max_epochs=3,log_every_n_steps=1)
else:
    trainer:pl.Trainer = pl.Trainer(accelerator='cpu',max_epochs=3,log_every_n_steps=1)


GPU available: True, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/sel/anaconda3/envs/ncem/lib/python3.8/site-packages/pytorch_lightning/trainer/trainer.py:1814: PossibleUserWarning: GPU available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='gpu', devices=1)`.
  rank_zero_warn(


In [9]:
trainer.fit(model,datamodule=dm)


  | Name        | Type            | Params
------------------------------------------------
0 | model_sigma | LinearSpatial   | 408   
1 | model_mu    | LinearSpatial   | 408   
2 | loss_module | GaussianNLLLoss | 0     
------------------------------------------------
816       Trainable params
0         Non-trainable params
816       Total params
0.003     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

/home/sel/anaconda3/envs/ncem/lib/python3.8/site-packages/pytorch_lightning/utilities/data.py:72: UserWarning: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 2. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.
  warning_cache.warn(


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]